# Python, Flask and Web-visualizations
## GSFC PUG - OCT 2015

Welcome, [Goddard Python User's Group](https://lists.nasa.gov/mailman/listinfo/gsfc-python-users)!  

This guide outlines the steps we took in our **October 6th, 2015 Meetup** to write a web application, for data visualization, in [Python](http://python.org/) using the [Flask](http://flask.pocoo.org/) microframework.

+ <h3>Setup</h3>
 + Hello World
 + URL Routing
 + Templates
+ <h3>User Inputs</h3>
 + URL Variable Inputs
 + Forms (Flask-WTForms)
+ <h3>Data Display</h3>
 + Static (matplotlib, ggplot, seaborn)
 + Interactive (bokeh, mpld3, plotly)
+ <h3>Upgaded Interface</h3>
 + Layouts (Bootstrap, Foundation)
 + Document Object Model (jQuery)
 + Call backs (AJAX, Databases)
 + Custom visualizations (d3.js, nvd3)


* Setup, User Inputs, and Static plotting all deal with the "Backend", or Server
 * The Server is where flask is very handy
* Interactive display with python packages begins the "Frontend", or Client
 * The Client is really where JavaScript is more useful

* What is flask?
 * Flask takes care of the "Web Server Gateway Interface" (WSGI)
 
<img src="imgs/wsgi_example.png"></img>

When a user makes a request in the client, `flask` will bring you back into the python program on the server side. 

If a user asks for a yearly average dataset, you can use `pandas` to provide the sampling, and `flask` again helps with sending the data back to the user. 

Anything you can do in python, you are suddenly able to interact with via the `flask` module. 

There are also `flask` add-ons, which handle even more amazing tasks, like local language translataion, thwarting hackers, and creating beautiful barebones websites in seconds. 

Let's get started!

# Follow Along

I've tried to orgainze all the codes here:

    git clone https://github.com/jakebrinkmann/2015-PUG-flask-data-vis.git 
    cd 2015-PUG-flask-data-vis
    

# Dependencies

Starting off, we will need:  

* python 2.7
* flask
* An up-to-date modern browser

I will use other packages along the way, but these are really all you need

# Assumptions

* You are using a UNIX-based OS
* You have not used flask before
* You have little or no web-developer experience

If you have used flask before, or know web development... let's make the next talk together!

---

# Getting Started

    git checkout version-0.1

 + Hello World
 + URL Routing
 + Templates

---



# Flask Hello World

    git checkout tags/v0.1.0

Flask is quick to get running

In [ ]:
from flask import Flask

app = Flask(__name__) # WSGI application

@app.route('/')
def hello_world():
    return 'Hello from Flask!'

if __name__ == '__main__':
    app.run()

Congratulations, your first web app! It will be running at http://localhost:5000 (only accessible by your machine)

Flask can generate HTML from within Python

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def show_html():
    return '<h1>Big heading</h1><p>And some <b>paragraph</b> text</p>'

if __name__ == '__main__':
    app.run()

# Debugging

    git checkout tags/v0.1.2
    
If you enable debug support the server will reload itself on code changes, and it will also provide you with a helpful debugger if things go wrong

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def create_error():
    return 10 + 'a'

if __name__ == '__main__':
    app.run(debug=True)

# <span style="color: red">Warning</span>

The interactive debugger allows the execution of arbitrary code.  
This makes it a major security risk and therefore it must never be used on production machines.


    app.run(debug=os.environ('DEBUG', false))
    

# Route Handling

    git checkout tags/v0.2.0

Flask can also handle more complex redirects, and request routes

In [ ]:
from flask import Flask, url_for

@app.route('/proj')
def projects():
    return '<h1>Project Page</h1><p><ul><li>PUG Data Visualization</li></ul></p>'


@app.route('/')
def hello_world():
    return '<a href="'+ url_for('projects') +'">Projects Page</a>'

if __name__ == '__main__':
    app.run()

# App Layout

    git checkout tags/v0.2.1
    
It is important to seperate the *presentation* from the *logic*


In [ ]:
! mkdir app
! mkdir app/static
! mkdir app/templates
! mkdir tmp
! touch app/__init__.py
! touch app/views.py
! touch run.py

    app/__init__.py

In [ ]:
from flask import Flask

app = Flask(__name__)
from app import views

    app/views.py

In [ ]:
from app import app

@app.route('/')
@app.route('/index')
def index():
    return "Hello, World!"

    run.py

In [ ]:
#!conda/bin/python
from app import app
app.run(debug=True)

# Templates

    git checkout version-0.5

In [ ]:
from app import app

@app.route('/')
@app.route('/index')
def index():
    user = {'nickname': 'Dr. Robert'}  # fake user
    return '''
<html>
  <head>
    <title>Home Page</title>
  </head>
  <body>
    <h1>Hello, ''' + user['nickname'] + '''</h1>
  </body>
</html>
'''

# Jinja2 Template Engine

To keep the logic of your application separate from the layout or presentation of your web pages things would be much better organized

Jinja Templates are great, and the syntax is fairly straight forward. 
It is used internally by both:

* flask
* Jekyll (ruby)

Useful for flask, but also useful for other HTML tasks, like sending emails with preset formating


Control statements

    app/templates/index.html

In [ ]:
<html>
  <head>
    {% if title %}
    <title>{{ title }} - PUG</title>
    {% else %}
    <title>2015-PUG-Data-Vis</title>
    {% endif %}
  </head>
  <body>
      <h1>Hello, {{ user.nickname }}!</h1>
  </body>
</html>

    app/views.py

In [ ]:
from flask import render_template
from app import app

@app.route('/')
@app.route('/index')
def index():
    user = {'nickname': 'Dr. Robert'} 
    return render_template('index.html',
                           title='Home',
                           user=user)

---

# User Inputs

    git checkout version-0.2

 + URL Variable Inputs
 + Forms (Flask-WTForms)
 
---

<img src="imgs/click_stream.png" style="align: center"></img>

# URL Variables

    git checkout version-0.6


Add variable parts to a URL.  
Optionall converters exist:

+ int -	accepts integers
+ float - like int but for floating point values
+ path - like the default but also accepts slashes

In [ ]:
@app.route('/user/<username>')
def show_user_profile(username):
    # show the user profile for that user
    return 'User %s' % username

@app.route('/post/<int:post_id>')
def show_post(post_id):
    # show the post with the given id, the id is an integer
    return 'Post %d' % post_id

# Forms


HTTP Methods


In [ ]:
@app.route('/login', methods=['POST', 'GET'])
def login():
    error = None
    if request.method == 'POST':
        if valid_login(request.form['username'],
                       request.form['password']):
            return log_the_user_in(request.form['username'])
        else:
            error = 'Invalid username/password'
    # the code below is executed if the request method
    # was GET or the credentials were invalid
    return render_template('login.html', error=error)

Beware hackers

In [1]:
from flask import Markup
Markup('<strong>Hello %s!</strong>') % '<blink>hacker</blink>'

Markup(u'<strong>Hello &lt;blink&gt;hacker&lt;/blink&gt;!</strong>')

---

# Data Display

 + Static (matplotlib, ggplot, seaborn)
 + Interactive (bokeh, mpld3, plotly)

---

# Static Plotting

+ matplotlib
+ ggplot
+ seaborn

In [ ]:
from flask import Flask, make_response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure


app = Flask(__name__)


@app.route('/plot.png')
def plot():
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)

    xs = range(100)
    ys = [random.randint(1, 50) for x in xs]

    axis.plot(xs, ys)
    canvas = FigureCanvas(fig)
    output = StringIO.StringIO()
    canvas.print_png(output)
    response = make_response(output.getvalue())
    response.mimetype = 'image/png'
    return response

# Interactive Plotting

+ bokeh
+ mpld3
+ plotly


In [ ]:
from bokeh.embed import components
from bokeh.plotting import figure

@app.route('/plot/')
def hello():
    fig = figure(title="Polynomial")
    fig.line(x, [i ** 2 for i in x], color=color, line_width=2)
    script, div = components(fig)
    return render_template(
        'bokeh.html',
        script=script,
        div=div,
    )

In [ ]:
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Bokeh Plot</title>
       
        <link rel="stylesheet" href="http://cdn.pydata.org/bokeh/release/bokeh-0.9.1.min.css" type="text/css" />
        <script type="text/javascript" src="http://cdn.pydata.org/bokeh/release/bokeh-0.9.1.min.js"></script>
  </head>
  <body>
      {{ div | safe }}
      {{ script | safe }}
  </body>
</html>

---

# Upgraded Interface

 + Layouts (Bootstrap, Foundation)
 + Document Object Model (jQuery)
 + Call backs (AJAX, Databases)
 + Custom visualizations (d3.js, nvd3)

---

# Javascript Plotting

+ d3
+ nvd3
+ dygraphs
+ Google Charts

In [ ]:
<div id="chart1">
    <svg></svg>
</div>

<script>
    // Wrapping in nv.addGraph allows for '0 timeout render', stores rendered charts in nv.graphs,
    // and may do more in the future... it's NOT required
    nv.addGraph(function() {

# Javascript Awesome

+ Bootstrap, Foundation, etc.
+ JQUERY
+ AJAX

In [ ]:
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
    


# Depolyment

Couple of options for deployment

+ In-App
+ pythonanywhere, heroku
+ Apache2, nginx, RPi server, etc.

# In-App

    app.run(port='0.0.0.0', debug=False)

# pythonanywhere, heroku

Leave out the `app.run()` call, and setup configurations as directed by the host

[pythonanywhere](https://www.pythonanywhere.com/details/flask_hosting)

[heroku](https://devcenter.heroku.com/articles/getting-started-with-python-o)



# Setting up Apache 

    FcgidIPCDir /tmp
    AddHandler fcgid-script .fcgi
    <VirtualHost *:80>
        DocumentRoot /var/www/datavis/app/static
        Alias /static /var/www/datavis/app/static
        ScriptAlias / /var/www/datavis/runp-mysql.fcgi/
    </VirtualHost>

---

# CONCLUSIONS ==================================

# Common Pitfalls

Things which seemed to pop up reliably to trip me up early on

## Flask

+ Using loops, other Jinja controls
+ Keeping logic seperate from UI/UX
+ Pre-process & ready the data

## Javascript

+ Javascript is complied language, and is getting faster
+ + Out-source some processes to the client browser
+ Imports can overwrite, or depend on, other imports
+ 

## Website

+ FLask is best for when you need the server backend
+ Jekyll for static webpages
+ Drupal for multi-user content

# References

[flask docs](http://flask.pocoo.org/docs/0.10/quickstart/)

[The Flask Mega-Tutorial](http://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world)
